# Feature_Engineering_Final_version
## IEEE Fraud Detection (Kaggle Current Competition)


Training Part:\
train_transaction\
train_identity


Testing Part:\
test_transaction\
test_identity

Dataset can be downloaded via:\
https://www.kaggle.com/c/ieee-fraud-detection/data



In [1]:
 # import basic package
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style

#Deal with warnings
import warnings
warnings.filterwarnings('ignore')
style.use('ggplot')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

#import my own packages
# import fraud_fun
# import myfunc_pd as mf


# from package import binary encoder
import category_encoders as ce

#import package for ML and feature engineering
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Loading data

In [3]:
# Read the trian and test dataset.
train_transaction = pd.read_csv('train_transaction.csv')
train_identity = pd.read_csv('train_identity.csv')
test_transaction = pd.read_csv('test_transaction.csv')
test_identity = pd.read_csv('test_identity.csv')

In [102]:
# Merge two dataset, 

train = train_transaction.merge(train_identity, how='left', on='TransactionID')
# train.head()
test = test_transaction.merge(test_identity, how='left', on='TransactionID')
# train.head()

In [103]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [104]:
test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Missing values



In [105]:

train_missing_cat = []
train_missing_num = []
test_missing_cat = []
test_missing_num = []
for col in train.columns:
    
    if train[col].dtypes == 'object':
        train_missing_cat.append(col)     
    else:
        train_missing_num.append(col)
        
for col_ in test.columns:
    
    if test[col_].dtypes == 'object':
        test_missing_cat.append(col_)     
    else:
        test_missing_num.append(col_)        

print(f'****** finish detection **********')
print(f'train_missing_cat has {len(train_missing_cat)} features')
print(f'train_missing_num has {len(train_missing_num)} features')
print(f'test_missing_cat has {len(test_missing_cat)} features')
print(f'test_missing_num has {len(test_missing_num)} features')


****** finish detection **********
train_missing_cat has 31 features
train_missing_num has 403 features
test_missing_cat has 31 features
test_missing_num has 402 features


In [106]:
for i in train_missing_num:
    train[i].fillna((train[i].mean()), inplace=True)
for i in train_missing_cat:
    train[i].fillna('NotFound', inplace=True)
    
for i in test_missing_num:
    test[i].fillna((test[i].mean()), inplace=True)
for i in test_missing_cat:
    test[i].fillna('NotFound', inplace=True)

In [107]:
train.drop(columns='TransactionID',inplace=True)
test.drop(columns='TransactionID',inplace=True)

In [108]:
train.shape, test.shape

((590540, 433), (506691, 432))

In [109]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

# Feature Engineering

In this section, we will maily focus on TransactionDT column, to generate some potential patterns.

Several randomly mean and std will be generated according to the feature in same sub_sections (V1-V399 etc.)

In [110]:
# Processing date from reference:
#https://www.kaggle.com/kevinbonnes/transactiondt-starting-at-2017-12-01


START_DATE = '2017-12-01'
# df = pd.concat([train_transaction, test_transaction], axis = 0, sort = False)
# df = train_transaction.merge(train_identity, how='left', on='TransactionID')
# Preprocess date column
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
train['TransactionDT'] = train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
test['TransactionDT'] = test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
print(train['TransactionDT'].head(10))
print(test['TransactionDT'].head(10))

0   2017-12-02 00:00:00
1   2017-12-02 00:00:01
2   2017-12-02 00:01:09
3   2017-12-02 00:01:39
4   2017-12-02 00:01:46
5   2017-12-02 00:01:50
6   2017-12-02 00:02:02
7   2017-12-02 00:02:09
8   2017-12-02 00:02:15
9   2017-12-02 00:02:16
Name: TransactionDT, dtype: datetime64[ns]
0   2018-07-02 00:00:24
1   2018-07-02 00:01:03
2   2018-07-02 00:01:50
3   2018-07-02 00:01:50
4   2018-07-02 00:01:57
5   2018-07-02 00:02:03
6   2018-07-02 00:02:30
7   2018-07-02 00:03:07
8   2018-07-02 00:03:25
9   2018-07-02 00:03:36
Name: TransactionDT, dtype: datetime64[ns]


In [111]:
# Apply the different aspect into the new columns according to the date format.
# df['Year'] = pd.to_datetime(df['TransactionDT']).dt.year
train['Month'] = pd.to_datetime(train['TransactionDT']).dt.month
train['Day'] = pd.to_datetime(train['TransactionDT']).dt.day
train['Hour'] = pd.to_datetime(train['TransactionDT']).dt.hour
train['Dayofweek'] = pd.to_datetime(train['TransactionDT']).dt.dayofweek

test['Month'] = pd.to_datetime(test['TransactionDT']).dt.month
test['Day'] = pd.to_datetime(test['TransactionDT']).dt.day
test['Hour'] = pd.to_datetime(test['TransactionDT']).dt.hour
test['Dayofweek'] = pd.to_datetime(test['TransactionDT']).dt.dayofweek
# put the TranscationDT column into drop_columns list

train.drop(columns='TransactionDT',inplace=True)
test.drop(columns='TransactionDT',inplace=True)
train.shape, test.shape

((590540, 436), (506691, 435))

In [112]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [113]:
# # Get the different numerical sub_features such as v_feature, id_feature

card_features = ['card1','card2','card3','card5']
c_features = [c for c in train.columns if 'C' in c and c != 'ProductCD' and c not in \
             train_missing_cat]
d_features = [d for d in train.columns if 'D' in d and (d != 'TransactionID')\
        and (d != 'TransactionDT') and (d != 'ProductCD')and (d != 'DeviceType')and\
         (d != 'DeviceInfo') and (d not in train_missing_cat)\
             and (d != 'Day') and (d != 'Dayofweek')]
v_features = [c for c in train.columns if c[0] == 'V' and c not in \
             train_missing_cat]
id_features = [c for c in train.columns if c[0:2] == 'id' and c not in \
             train_missing_cat]

In [114]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [115]:
## Apply the mean and std for each sub_features 
train['card_features_mean'] = train[card_features].mean(axis=1)
train['card_features_std'] = train[card_features].std(axis=1)
train['c_features_mean'] = train[c_features].mean(axis=1)
train['c_features_std'] = train[c_features].std(axis=1)
train['d_features_mean'] = train[d_features].mean(axis=1)
train['d_features_std'] = train[d_features].std(axis=1)
train['v_features_mean'] = train[v_features].mean(axis=1)
train['v_features_std'] = train[v_features].std(axis=1)
train['id_features_mean'] = train[id_features].mean(axis=1)
train['id_features_std'] = train[id_features].std(axis=1)

## Apply the mean and std for each sub_features 
test['card_features_mean'] = test[card_features].mean(axis=1)
test['card_features_std'] = test[card_features].std(axis=1)
test['c_features_mean'] = test[c_features].mean(axis=1)
test['c_features_std'] = test[c_features].std(axis=1)
test['d_features_mean'] = test[d_features].mean(axis=1)
test['d_features_std'] = test[d_features].std(axis=1)
test['v_features_mean'] = test[v_features].mean(axis=1)
test['v_features_std'] = test[v_features].std(axis=1)
test['id_features_mean'] = test[id_features].mean(axis=1)
test['id_features_std'] = test[id_features].std(axis=1)

train.shape, test.shape

((590540, 446), (506691, 445))

In [116]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [117]:
#try to use card features
for col in ['card1','card2','card4','card5','card6']:
    # we are just taking a mean for each group and diving it with the each group Transaction amount to get more information
    # and also std for each group 
    train['Transactionamt_mean_'+str(col)]=(train['TransactionAmt']/train.groupby(col)['TransactionAmt'].transform('mean'))
    train['Transactionamt_std_'+str(col)]=(train['TransactionAmt']/train.groupby(col)['TransactionAmt'].transform('std'))
    test['Transactionamt_mean_'+str(col)]=(test['TransactionAmt']/test.groupby(col)['TransactionAmt'].transform('mean'))
    test['Transactionamt_std_'+str(col)]=(test['TransactionAmt']/test.groupby(col)['TransactionAmt'].transform('std'))

# Drop the features which will cause errors for future Normalisation 
train.drop(columns=['Transactionamt_std_card1','Transactionamt_std_card5'],inplace=True)
test.drop(columns=['Transactionamt_std_card1','Transactionamt_std_card5'],inplace=True)
train.shape, test.shape

((590540, 454), (506691, 453))

In [118]:
test['Transactionamt_std_card2'].fillna(method='ffill',inplace=True)
test['Transactionamt_std_card6'].fillna(method='ffill',inplace=True)

In [119]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [120]:
# feature engineering Id_cols 
# Part from Reference :https://www.kaggle.com/kabure/almost-complete-feature-engineering-ieee-data
# Set-up new column to get the info from Deviceinfo, add label for them./

train.loc[train['DeviceInfo'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
train.loc[train['DeviceInfo'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
train.loc[train['DeviceInfo'].str.contains('GT-', na=False), 'device_name'] = 'Samsung' 
train.loc[train['DeviceInfo'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
train.loc[train['DeviceInfo'].str.contains('Moto', na=False), 'device_name'] = 'Motorola' 
train.loc[train['DeviceInfo'].str.contains('moto', na=False), 'device_name'] = 'Motorola' 
train.loc[train['DeviceInfo'].str.contains('LG-', na=False), 'device_name'] = 'LG' 
train.loc[train['DeviceInfo'].str.contains('rv:', na=False), 'device_name'] = 'RV' 
train.loc[train['DeviceInfo'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei' 
train.loc[train['DeviceInfo'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei' 
train.loc[train['DeviceInfo'].str.contains('-L', na=False), 'device_name'] = 'Huawei' 
train.loc[train['DeviceInfo'].str.contains('Blade', na=False), 'device_name'] = 'ZTE' 
train.loc[train['DeviceInfo'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE' 
train.loc[train['DeviceInfo'].str.contains('Linux', na=False), 'device_name'] = 'Linux' 
train.loc[train['DeviceInfo'].str.contains('XT', na=False), 'device_name'] = 'Sony' 
train.loc[train['DeviceInfo'].str.contains('HTC', na=False), 'device_name'] = 'HTC' 
train.loc[train['DeviceInfo'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'
train.loc[train['DeviceInfo'].str.contains('Not', na=True), 'device_name'] = 'NotFound'

test.loc[test['DeviceInfo'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
test.loc[test['DeviceInfo'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
test.loc[test['DeviceInfo'].str.contains('GT-', na=False), 'device_name'] = 'Samsung' 
test.loc[test['DeviceInfo'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
test.loc[test['DeviceInfo'].str.contains('Moto', na=False), 'device_name'] = 'Motorola' 
test.loc[test['DeviceInfo'].str.contains('moto', na=False), 'device_name'] = 'Motorola' 
test.loc[test['DeviceInfo'].str.contains('LG-', na=False), 'device_name'] = 'LG' 
test.loc[test['DeviceInfo'].str.contains('rv:', na=False), 'device_name'] = 'RV' 
test.loc[test['DeviceInfo'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei' 
test.loc[test['DeviceInfo'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei' 
test.loc[test['DeviceInfo'].str.contains('-L', na=False), 'device_name'] = 'Huawei' 
test.loc[test['DeviceInfo'].str.contains('Blade', na=False), 'device_name'] = 'ZTE' 
test.loc[test['DeviceInfo'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE' 
test.loc[test['DeviceInfo'].str.contains('Linux', na=False), 'device_name'] = 'Linux' 
test.loc[test['DeviceInfo'].str.contains('XT', na=False), 'device_name'] = 'Sony' 
test.loc[test['DeviceInfo'].str.contains('HTC', na=False), 'device_name'] = 'HTC' 
test.loc[test['DeviceInfo'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'
test.loc[test['DeviceInfo'].str.contains('Not', na=True), 'device_name'] = 'NotFound'
# Drop the previous column.
train.drop(columns='DeviceInfo',inplace=True)
test.drop(columns='DeviceInfo',inplace=True)

train.shape, test.shape

((590540, 454), (506691, 453))

In [121]:
test['device_name'].fillna('NotFound',inplace=True)
train['device_name'].fillna('NotFound',inplace=True)

In [122]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [123]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

In [124]:
# feature Enginnering based on ProductCD because it has only 4 levels
# and also for P_emaildomain ,R_emaildomain,DeviceType
#DeviceInfo,id_15,id_23,id_30,id_31,id_34

for col in ['ProductCD', 'P_emaildomain','R_emaildomain','DeviceType','device_name','id_15','id_23','id_30','id_34']:
    train['Transactionamt_mean_'+str(col)]=(train['TransactionAmt']/train.groupby(col)['TransactionAmt'].transform('mean'))
    train['Transactionamt_std_'+str(col)]=(train['TransactionAmt']/train.groupby(col)['TransactionAmt'].transform('std'))
    
    test['Transactionamt_mean_'+str(col)]=(test['TransactionAmt']/test.groupby(col)['TransactionAmt'].transform('mean'))
    test['Transactionamt_std_'+str(col)]=(test['TransactionAmt']/test.groupby(col)['TransactionAmt'].transform('std'))
# Part Reference:  https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm-corrected

# feature engineering with TransactionAmt

train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')
#feature engineering with id_02
train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')
#feature engineering with D_15
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')
train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')
#feature engineering with id_02
test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')
#feature engineering with D_15
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')
test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')
#Reference: https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499

for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails)
    test[c + '_bin'] = test[c].map(emails)
    
    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])
    
    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [125]:
train['D15_to_mean_addr1'].fillna(train['D15_to_mean_addr1'].mean(),inplace=True)
train['Transactionamt_std_id_30'].fillna(train['Transactionamt_std_id_30'].mean(),inplace=True)
train['P_emaildomain_bin'].fillna('NotFound',inplace=True)
train['R_emaildomain_bin'].fillna('NotFound',inplace=True)


test['D15_to_mean_addr1'].fillna(test['D15_to_mean_addr1'].mean(),inplace=True)
test['Transactionamt_std_id_34'].fillna(test['Transactionamt_std_id_34'].mean(),inplace=True)
test['P_emaildomain_bin'].fillna('NotFound',inplace=True)
test['R_emaildomain_bin'].fillna('NotFound',inplace=True)

In [126]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [127]:
# New feature - log of transaction amount. ()
train['TransactionAmt_Log'] = np.log(train['TransactionAmt'])
test['TransactionAmt_Log'] = np.log(test['TransactionAmt'])

In [128]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [129]:
# Drop the features which will cause errors for future Normalisation 
train.drop(columns=['Transactionamt_std_P_emaildomain'],inplace=True)
test.drop(columns=['Transactionamt_std_P_emaildomain'],inplace=True)

train.shape, test.shape

((590540, 483), (506691, 482))

In [ ]:
# # Bakcup
# train.to_csv('FullData_train.csv')

In [ ]:
# # Bakcup
# test.to_csv('FullData_test.csv')

# Encoder

In [ ]:
# train = pd.read_csv('FullData_train.csv')
# train.drop(columns='Unnamed: 0',inplace=True)
# # train.head()
# test = pd.read_csv('FullData_test.csv')
# test.drop(columns='Unnamed: 0',inplace=True)
# # train.head()

# train.shape, test.shape

In [130]:

train_missing_cat = []
train_missing_num = []
test_missing_cat = []
test_missing_num = []
for col in train.columns:
    
    if train[col].dtypes == 'object':
        train_missing_cat.append(col)     
    else:
        train_missing_num.append(col)
        
for col_ in test.columns:
    
    if test[col_].dtypes == 'object':
        test_missing_cat.append(col_)     
    else:
        test_missing_num.append(col_)        
print(f'****** finish detection **********')
print(f'train_missing_cat has {len(train_missing_cat)} features')
print(f'train_missing_num has {len(train_missing_num)} features')
print(f'test_missing_cat has {len(test_missing_cat)} features')
print(f'test_missing_num has {len(test_missing_num)} features')


****** finish detection **********
train_missing_cat has 35 features
train_missing_num has 448 features
test_missing_cat has 35 features
test_missing_num has 447 features


In [131]:
# train_missing_cat.remove('DeviceInfo')
large_col_cat_train = []
small_col_cat_train = []

for feature in train_missing_cat:
    if train[feature].nunique() > 5:
        large_col_cat_train.append(feature)
    else: 
        small_col_cat_train.append(feature)
print(f'large_col_cat_train: {large_col_cat_train}')
print(f'small_col_cat_train: {small_col_cat_train}')

large_col_cat_train: ['P_emaildomain', 'R_emaildomain', 'id_30', 'id_31', 'id_33', 'device_name', 'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix']
small_col_cat_train: ['ProductCD', 'card4', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType']


In [132]:
# test_missing_cat.remove('DeviceInfo')
large_col_cat_test = []
small_col_cat_test = []

for feature in test_missing_cat:
    if test[feature].nunique() > 5:
        large_col_cat_test.append(feature)
    else: 
        small_col_cat_test.append(feature)
print(f'large_col_cat_test: {large_col_cat_test}')
print(f'small_col_cat_test: {small_col_cat_test}')

large_col_cat_test: ['P_emaildomain', 'R_emaildomain', 'id_30', 'id_31', 'id_33', 'device_name', 'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix']
small_col_cat_test: ['ProductCD', 'card4', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType']


In [133]:
# For large_col_cat, the binary labelencoder will be applied.
df_binary = binary_encoder(train,large_col_cat_train)
train = pd.concat([train,df_binary],axis=1)


#For small_col_cat, the get_dummies will be applied.
df_dummies = get_dummies_label(train,small_col_cat_train)
train = pd.concat([train,df_dummies],axis=1)


In [134]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [135]:
# For large_col_cat, the binary labelencoder will be applied.
df_binary = binary_encoder(test,large_col_cat_test)
test = pd.concat([test,df_binary],axis=1)


#For small_col_cat, the get_dummies will be applied.
df_dummies = get_dummies_label(test,small_col_cat_test)
test = pd.concat([test,df_dummies],axis=1)


train.shape, test.shape

((590540, 606), (506691, 602))

In [136]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [137]:
train.drop(columns=large_col_cat_train, inplace=True)
train.drop(columns=small_col_cat_train, inplace=True)

test.drop(columns=large_col_cat_test, inplace=True)
test.drop(columns=small_col_cat_test, inplace=True)

train.shape, test.shape

((590540, 571), (506691, 567))

In [138]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

# Normalization

In [139]:
array = train.values
# create scaler
scaler = MinMaxScaler()

# fit and transform in one step
df2 = scaler.fit_transform(array)

train = pd.DataFrame(df2, columns=train.columns)


In [140]:
array_ = test.values
# create scaler
scaler = MinMaxScaler()

# fit and transform in one step
df_test = scaler.fit_transform(array_)

test = pd.DataFrame(df_test, columns=test.columns)

train.shape, test.shape

((590540, 571), (506691, 567))

In [141]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [142]:
# Bakcup
train.to_csv('FullData_nor_train.csv')
test.to_csv('FullData_nor_test.csv')

In [143]:
diff_cols = [c for c in train.columns if c not in test.columns]


In [144]:
diff_cols

['isFraud',
 'card6_debit or credit',
 'id_34_match_status:-1',
 'id_34_match_status:0']

In [148]:
test['card6_debit or credit'] = 0
test['id_34_match_status:-1'] = 0
test['id_34_match_status:0'] = 0

In [150]:
test.to_csv('FullData_nor_test.csv')

In [151]:
train.isna().any().sum(), test.isna().any().sum()

(0, 0)

In [152]:
train.shape, test.shape

((590540, 571), (506691, 570))

# Func

In [2]:
def binary_encoder(DataFrame, col):

    # Using the binary encoder to deal with large_col_cat:

    df = DataFrame[col]

    # Calling encoder function.
    encoder = ce.BinaryEncoder(cols=col)
    df_binary = encoder.fit_transform(df)
    return df_binary
    # Concat the features with previous dataset
#     DataFrame = pd.concat([DataFrame,df_binary],axis=1)
#     DataFrame.drop(columns=col, inplace=True)
    print(f'************** Finish binary_encoder *****************')
    
    

def get_dummies_label(DataFrame, col):
    # Using dummies to deal with the small amount of unique values in small_col_cat
    df_small = DataFrame[col]
    # Get dummies
    df_dummies = pd.get_dummies(df_small, prefix_sep='_', drop_first=True)
    return df_dummies
    # Concat the features with previous dataset
#     DataFrame = pd.concat([DataFrame,df_dummies],axis=1)
#     DataFrame.drop(columns=col, inplace=True)
    
    print(f'************** Finish get_dummies_label *****************')